In [1]:
# Import necessary modules
import pyspark  # Only run this after findspark.init() if you need to initialize Spark in your environment
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf

# Initialize or retrieve the Spark session
# Ensure SparkSession is already created and available as 'spark'
# If 'spark' is not available, you need to create a SparkSession like below:
# spark = SparkSession.builder.appName("exampleApp").getOrCreate()

# Get the number of cores being used
cores = spark._jsc.sc().getExecutorMemoryStatus().keySet().size()

# Get the application ID of the current Spark application
appid = spark._jsc.sc().applicationId()

# Print the number of cores and the application ID
print("You are working with", cores, "core(s) on appid: ", appid)

# Return the Spark session object (this will display the object in an interactive environment)
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,application_1717027860258_0002,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

You are working with 4 core(s) on appid:  application_1717027860258_0002

In [2]:
# Install pandas in EMR cluster
sc.install_pypi_package("pandas")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Attempting uninstall: python-dateutil
    Found existing installation: python-dateutil 2.8.1
    Not uninstalling python-dateutil at /usr/lib/python3.9/site-packages, outside environment /mnt/yarn/usercache/livy/appcache/application_1717027860258_0002/container_1717027860258_0002_01_000001/tmp/spark-83a99d2c-a77f-4b61-a739-3ef3cb4b08de
    Can't uninstall 'python-dateutil'. No files were found to uninstall.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
awscli 2.15.30 requires python-dateutil<=2.8.2,>=2.1, but you have python-dateutil 2.9.0.post0 which is incompatible.

In [3]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

# Define schema for each file
ratings_schema = "UserID INT, MovieID INT, Rating FLOAT, Timestamp LONG"
movies_schema = "MovieID INT, Title STRING, Genres STRING"

# Read the dataset from S3
ratings = spark.read.csv("s3://assignment6neha/ratings.dat", sep="::", schema=ratings_schema)
movies = spark.read.csv("s3://assignment6neha/movies.dat", sep="::", schema=movies_schema)




VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
from pyspark.sql.functions import col

toy_story_ratings = ratings.filter(col("MovieID") == 1).select("UserID", col("MovieID").alias("MovieID1"), col("Rating").alias("Rating1"))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
toy_story_ratings.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+-------+
|UserID|MovieID1|Rating1|
+------+--------+-------+
|     5|       1|    1.0|
|    14|       1|    3.0|
|    18|       1|    3.0|
|    23|       1|    5.0|
|    24|       1|    5.0|
|    30|       1|    5.0|
|    33|       1|    4.0|
|    34|       1|    3.0|
|    35|       1|    3.0|
|    36|       1|    4.0|
|    43|       1|    4.0|
|    47|       1|    3.0|
|    56|       1|    4.5|
|    57|       1|    4.0|
|    58|       1|    4.0|
|    64|       1|    4.0|
|    65|       1|    3.0|
|    76|       1|    4.0|
|    77|       1|    4.0|
|    83|       1|    3.0|
+------+--------+-------+
only showing top 20 rows

In [8]:
joined_ratings = ratings.alias("r1").join(toy_story_ratings.alias("r2"), "UserID", "inner") \
    .filter(col("r1.MovieID") != 1)  # Exclude Toy Story itself from the join

# Select relevant columns
combined_ratings = joined_ratings.select("UserID", "r1.MovieID", "MovieID1", "r1.Rating", "Rating1")

# Rename columns for clarity
combined_ratings = combined_ratings.withColumnRenamed("MovieID", "MovieID2").withColumnRenamed("r1.Rating", "Rating2")

# Show the resulting DataFrame
combined_ratings.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+--------+------+-------+
|UserID|MovieID2|MovieID1|Rating|Rating1|
+------+--------+--------+------+-------+
|   148|      17|       1|   5.0|    5.0|
|   148|      21|       1|   4.0|    5.0|
|   148|      29|       1|   5.0|    5.0|
|   148|      32|       1|   5.0|    5.0|
|   148|      34|       1|   5.0|    5.0|
|   148|      43|       1|   4.0|    5.0|
|   148|      47|       1|   4.0|    5.0|
|   148|      50|       1|   5.0|    5.0|
|   148|      70|       1|   5.0|    5.0|
|   148|     112|       1|   4.0|    5.0|
|   148|     123|       1|   5.0|    5.0|
|   148|     141|       1|   4.0|    5.0|
|   148|     150|       1|   5.0|    5.0|
|   148|     161|       1|   4.0|    5.0|
|   148|     180|       1|   4.0|    5.0|
|   148|     260|       1|   5.0|    5.0|
|   148|     266|       1|   2.0|    5.0|
|   148|     272|       1|   4.0|    5.0|
|   148|     293|       1|   5.0|    5.0|
|   148|     296|       1|   5.0|    5.0|
+------+--------+--------+------+-

In [13]:
from pyspark.sql.functions import collect_list

# Group by MovieID1 and MovieID2 pairs and collect the ratings into arrays
grouped_df = combined_ratings.groupby("MovieID1", "MovieID2").agg(collect_list("Rating").alias("Ratings1"),
                                                                 collect_list("Rating1").alias("Ratings2"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
grouped_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------------------+--------------------+
|MovieID1|MovieID2|            Ratings1|            Ratings2|
+--------+--------+--------------------+--------------------+
|       1|       9|[2.0, 1.0, 3.0, 3...|[4.0, 3.0, 5.0, 3...|
|       1|      20|[2.0, 0.5, 3.0, 1...|[5.0, 3.0, 3.5, 3...|
|       1|     124|[3.0, 5.0, 4.0, 3...|[4.0, 3.0, 4.0, 5...|
|       1|     199|[5.0, 4.5, 3.0, 5...|[3.5, 5.0, 4.0, 2...|
|       1|     204|[3.0, 4.0, 3.0, 1...|[5.0, 3.0, 4.0, 3...|
|       1|     213|[4.0, 4.0, 2.0, 4...|[5.0, 4.0, 5.0, 4...|
|       1|     269|[3.0, 3.5, 3.0, 4...|[3.0, 4.0, 2.0, 5...|
|       1|     349|[4.0, 3.0, 4.0, 3...|[5.0, 3.0, 4.0, 3...|
|       1|     381|[2.5, 4.0, 3.0, 2...|[4.0, 4.0, 4.0, 3...|
|       1|     398|[3.0, 2.0, 4.0, 3...|[3.0, 3.0, 4.0, 5...|
|       1|     433|[2.0, 3.0, 4.0, 2...|[4.0, 4.0, 5.0, 3...|
|       1|     449|[4.0, 5.0, 2.0, 3...|[4.0, 2.0, 4.5, 4...|
|       1|     527|[3.0, 5.0, 5.0, 5...|[3.0, 4.0, 4.0, 4...|
|       

In [51]:
from pyspark.sql.functions import arrays_zip, col

# Combine the Ratings1 and Ratings2 arrays into an array of structs
ratings_array = arrays_zip(col("Ratings1"), col("Ratings2")).alias("Ratings")

# Select MovieID1, MovieID2, and the combined Ratings array
result_df = grouped_df.select("MovieID1", "MovieID2", ratings_array)

# Show the result
result_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------------------+
|MovieID1|MovieID2|             Ratings|
+--------+--------+--------------------+
|       1|       9|[{3.0, 3.0}, {2.5...|
|       1|      20|[{4.0, 3.5}, {4.0...|
|       1|     124|[{5.0, 4.0}, {4.0...|
|       1|     199|[{5.0, 4.0}, {4.0...|
|       1|     204|[{4.0, 3.0}, {5.0...|
|       1|     213|[{3.0, 4.0}, {4.0...|
|       1|     269|[{5.0, 5.0}, {3.0...|
|       1|     349|[{4.0, 5.0}, {4.0...|
|       1|     381|[{5.0, 4.0}, {4.0...|
|       1|     398|[{4.0, 4.0}, {4.0...|
|       1|     433|[{3.0, 4.0}, {3.0...|
|       1|     449|[{3.5, 3.0}, {4.0...|
|       1|     527|[{4.0, 4.0}, {2.0...|
|       1|     549|[{5.0, 3.0}, {5.0...|
|       1|     619|[{3.0, 3.0}, {2.0...|
|       1|     636|[{1.0, 2.0}, {3.0...|
|       1|     655|[{2.0, 2.0}, {2.0...|
|       1|     666|[{3.0, 4.0}, {3.0...|
|       1|     670|[{1.5, 3.0}, {5.0...|
|       1|     685|[{2.0, 4.0}, {5.0...|
+--------+--------+--------------------+
only showing top

In [20]:
result_df.dtypes

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[('MovieID1', 'int'), ('MovieID2', 'int'), ('Ratings', 'array<struct<Ratings1:float,Ratings2:float>>')]

In [53]:
from math import sqrt
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType

# Define the cosine_similarity function
def cosine_similarity(ratings):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    
    # Iterate over each rating pair
    for ratingPair in ratings:
        ratingX, ratingY = ratingPair
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    # Calculate cosine similarity
    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)
    score = 0 if denominator == 0 else numerator / denominator
    
    # Return the result as a struct
    return (score, numPairs)

# Define the schema for the struct
schema = StructType([
    StructField("Score", FloatType(), nullable=False),
    StructField("NumPairs", IntegerType(), nullable=False)
])

# Register the cosine_similarity function as a UDF
cosine_similarity_udf = udf(cosine_similarity, schema)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [54]:
result_df1 = result_df
result_df1 = result_df1.withColumn("CosineSimilarity", cosine_similarity_udf("Ratings"))

# Extract the "Score" and "NumPairs" fields from the "CosineSimilarity" struct column
result_df1 = result_df1.withColumn("Score", result_df1["CosineSimilarity"]["Score"])
result_df1 = result_df1.withColumn("NumPairs", result_df1["CosineSimilarity"]["NumPairs"])

# Drop the "CosineSimilarity" column as it's no longer needed
#result_df1 = result_df1.drop("CosineSimilarity")

# Show the updated DataFrame
result_df1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------------------+------------------+----------+--------+
|MovieID1|MovieID2|             Ratings|  CosineSimilarity|     Score|NumPairs|
+--------+--------+--------------------+------------------+----------+--------+
|       1|       9|[{1.5, 2.5}, {4.0...|{0.93348676, 1637}|0.93348676|    1637|
|       1|      20|[{3.0, 4.0}, {3.0...|  {0.930877, 1308}|  0.930877|    1308|
|       1|     124|[{5.0, 5.0}, {3.0...|  {0.9424575, 132}| 0.9424575|     132|
|       1|     199|[{4.0, 4.0}, {2.0...| {0.94670266, 496}|0.94670266|     496|
|       1|     204|[{3.0, 4.0}, {2.0...| {0.9255814, 2277}| 0.9255814|    2277|
|       1|     213|[{3.5, 4.0}, {3.0...| {0.95106447, 531}|0.95106447|     531|
|       1|     269|[{2.0, 4.0}, {3.0...| {0.94344777, 224}|0.94344777|     224|
|       1|     349|[{2.5, 4.0}, {5.0...| {0.9611848, 7993}| 0.9611848|    7993|
|       1|     381|[{3.0, 4.0}, {3.0...|{0.94690937, 1373}|0.94690937|    1373|
|       1|     398|[{3.0, 5.0}, {1.0...|

In [61]:
filtered_df = result_df1.filter((result_df1["NumPairs"] > 10) & (result_df1["Score"] > 0.95))
filtered_df = filtered_df.orderBy("Score", ascending=False)

top_10_df = filtered_df.limit(10)

# Show the top 10 similar movies
top_10_df.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------------------+----------------+----------+--------+
|MovieID1|MovieID2|             Ratings|CosineSimilarity|     Score|NumPairs|
+--------+--------+--------------------+----------------+----------+--------+
|       1|    6336|[{4.0, 4.0}, {3.5...|{0.99469984, 13}|0.99469984|      13|
|       1|   41831|[{3.0, 3.0}, {4.5...|{0.99301666, 17}|0.99301666|      17|
|       1|    8422|[{4.0, 4.5}, {3.5...|{0.99198604, 17}|0.99198604|      17|
|       1|   55895|[{3.0, 3.5}, {3.5...| {0.9902073, 15}| 0.9902073|      15|
|       1|    7273|[{3.0, 5.0}, {2.0...| {0.9896856, 11}| 0.9896856|      11|
|       1|   25931|[{3.0, 5.0}, {3.5...|{0.98955035, 12}|0.98955035|      12|
|       1|   32781|[{4.0, 5.0}, {3.0...| {0.9892781, 24}| 0.9892781|      24|
|       1|   62801|[{4.0, 4.5}, {3.5...|{0.98892105, 17}|0.98892105|      17|
|       1|   26996|[{3.5, 4.5}, {2.5...|{0.98886037, 11}|0.98886037|      11|
|       1|   39896|[{3.0, 4.5}, {2.5...| {0.9883148, 12}| 0.9883

In [65]:
from pyspark.sql.functions import col

# Perform two join operations to replace MovieID1 and MovieID2 with movie names
result_df_with_names = top_10_df.join(movies.withColumnRenamed("MovieID", "MovieID1").alias("movies1"), "MovieID1") \
    .join(movies.withColumnRenamed("MovieID", "MovieID2").alias("movies2"), "MovieID2") \
    .select(col("movies1.Title").alias("Movie1"), col("movies2.Title").alias("Movie2"), "Ratings", "CosineSimilarity", "Score", "NumPairs")

# Show the DataFrame with movie names
result_df_with_names.show()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+--------------------+----------------+----------+--------+
|          Movie1|              Movie2|             Ratings|CosineSimilarity|     Score|NumPairs|
+----------------+--------------------+--------------------+----------------+----------+--------+
|Toy Story (1995)|South Central (1992)|[{2.5, 4.0}, {3.5...| {0.9883148, 12}| 0.9883148|      12|
|Toy Story (1995)|   Safe House (1998)|[{3.5, 4.5}, {2.5...|{0.98886037, 11}|0.98886037|      11|
|Toy Story (1995)|Lone Wolf and Cub...|[{3.5, 3.5}, {4.5...|{0.98892105, 17}|0.98892105|      17|
|Toy Story (1995)|       Hawaii (1966)|[{3.0, 4.5}, {3.0...| {0.9892781, 24}| 0.9892781|      24|
|Toy Story (1995)|  Road to Rio (1947)|[{3.5, 5.0}, {4.0...|{0.98955035, 12}|0.98955035|      12|
|Toy Story (1995)|Piece of the Acti...|[{3.5, 5.0}, {2.5...| {0.9896856, 11}| 0.9896856|      11|
|Toy Story (1995)|Desperate Hours, ...|[{4.0, 4.0}, {4.0...| {0.9902073, 15}| 0.9902073|      15|
|Toy Story (1995)|  

In [66]:
result_df_with_names.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10

In [68]:
from pyspark.sql.functions import col

# Drop the columns "CosineSimilarity" and "NumPairs"
result_df_with_names_dropped = result_df_with_names.drop("CosineSimilarity", "NumPairs", "Ratings")

# Rename the columns "Movie1" and "Movie2"
result_df_with_names_renamed = result_df_with_names_dropped \
    .withColumnRenamed("Movie1", "Movie Name") \
    .withColumnRenamed("Movie2", "Similar Movies")

# Show the DataFrame with the desired column names
result_df_with_names_renamed.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+----------+
|      Movie Name|      Similar Movies|     Score|
+----------------+--------------------+----------+
|Toy Story (1995)|South Central (1992)| 0.9883148|
|Toy Story (1995)|   Safe House (1998)|0.98886037|
|Toy Story (1995)|Lone Wolf and Cub...|0.98892105|
|Toy Story (1995)|       Hawaii (1966)| 0.9892781|
|Toy Story (1995)|  Road to Rio (1947)|0.98955035|
|Toy Story (1995)|Piece of the Acti...| 0.9896856|
|Toy Story (1995)|Desperate Hours, ...| 0.9902073|
|Toy Story (1995)|    Kings Row (1942)|0.99198604|
|Toy Story (1995)|They Died with Th...|0.99301666|
|Toy Story (1995)|Marooned in Iraq ...|0.99469984|
+----------------+--------------------+----------+

In [69]:
from pyspark.sql.functions import desc

# Sort the DataFrame in descending order based on the "Score" column
result_df_with_names_sorted = result_df_with_names_renamed.orderBy(desc("Score"))

# Show the sorted DataFrame
result_df_with_names_sorted.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+--------------------+----------+
|      Movie Name|      Similar Movies|     Score|
+----------------+--------------------+----------+
|Toy Story (1995)|Marooned in Iraq ...|0.99469984|
|Toy Story (1995)|They Died with Th...|0.99301666|
|Toy Story (1995)|    Kings Row (1942)|0.99198604|
|Toy Story (1995)|Desperate Hours, ...| 0.9902073|
|Toy Story (1995)|Piece of the Acti...| 0.9896856|
|Toy Story (1995)|  Road to Rio (1947)|0.98955035|
|Toy Story (1995)|       Hawaii (1966)| 0.9892781|
|Toy Story (1995)|Lone Wolf and Cub...|0.98892105|
|Toy Story (1995)|   Safe House (1998)|0.98886037|
|Toy Story (1995)|South Central (1992)| 0.9883148|
+----------------+--------------------+----------+